In [1]:
#Importing required libraries

import matplotlib.pyplot as plt
import seaborn as sns

import os
import gc

from sklearn.model_selection import train_test_split


import tensorflow as tf
from tqdm.autonotebook import tqdm

import numpy as np
import pandas as pd 

from keras import Sequential
from keras.callbacks import EarlyStopping

from keras.optimizers import Adam, SGD
from keras.callbacks import ReduceLROnPlateau
from keras.layers import Flatten,Dense,BatchNormalization,Activation,Dropout
from keras.layers import Lambda, Input, GlobalAveragePooling2D,BatchNormalization
from keras.utils import to_categorical
# from keras import regularizers
from tensorflow.keras.models import Model


from tensorflow.keras.preprocessing.image import load_img

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/tmp/ipykernel_34/2143223839.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [5]:
labels = pd.read_csv('/kaggle/input/dog-breed-identification/labels.csv')
labels.head()

,id,breed
0,000bec180eb18c7604dcecc8fe0dba07,boston_bull
1,001513dfcb2ffafc82cccf4d8bbaba97,dingo
2,001cdf01b096e06d78e9e5112d419397,pekinese
3,00214f311d5d2247d5dfe4fe24b2303d,bluetick
4,0021f9ceb3235effd7fcde7f7538ed62,golden_retriever


In [6]:
import os
if len(os.listdir('/kaggle/input/dog-breed-identification/train/')) == len(labels['id']):
    print('Number of file matches number of actual images!')
else:
    print('Number of file doesnot matches number of actual images!!')

Number of file matches number of actual images!


In [8]:
# #Create list of alphabetically sorted labels.
# classes = sorted(list(set(labels['breed'])))
# n_classes = len(classes)
# print('Total unique breed {}'.format(n_classes))


# #Map each label string to an integer label.
# class_to_num = dict(zip(classes, range(n_classes)))
# class_to_num

In [9]:
input_shape = (331,331,3)


def images_to_array(directory, label_dataframe, target_size = input_shape):
    
    image_labels = label_dataframe['breed']
    images = np.zeros([len(label_dataframe), target_size[0], target_size[1], target_size[2]],dtype=np.uint8) #as we have huge data and limited ram memory. uint8 takes less memory
    y = np.zeros([len(label_dataframe),1],dtype = np.uint8)
    
    for ix, image_name in enumerate(tqdm(label_dataframe['id'].values)):
        img_dir = os.path.join(directory, image_name + '.jpg')
        img = load_img(img_dir, target_size = target_size)
        images[ix] = img
        del img
        
        dog_breed = image_labels[ix]
        y[ix] = class_to_num[dog_breed]
    
    y = to_categorical(y)
    
    return images, y

In [10]:
import time 
t = time.time()

X, y = images_to_array('/kaggle/input/dog-breed-identification/train', labels[:])

print('runtime in seconds: {}'.format(time.time() - t))

  0%|          | 0/10222 [00:00<?, ?it/s]

runtime in seconds: 100.04567861557007


In [18]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42)

print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(7155, 331, 331, 3) (7155, 120)
(3067, 331, 331, 3) (3067, 120)


In [19]:
#Learning Rate Annealer
lrr= ReduceLROnPlateau(monitor='val_acc', factor=.01, patience=3, min_lr=1e-5,verbose = 1)

#Prepare call backs
EarlyStop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

In [20]:
# Hyperparameters
batch_size= 128
epochs=15
learn_rate=.001
sgd=SGD(learning_rate=learn_rate, momentum=0.9, nesterov=False)
adam=Adam(learning_rate=learn_rate, beta_1=0.9, beta_2=0.999, epsilon=None,  amsgrad=False)

In [21]:
#function to extract features from the dataset by a given pretrained model
img_size = (331,331,3)

def get_features(model_name, model_preprocessor, input_size, data):

    input_layer = Input(input_size)
    preprocessor = Lambda(model_preprocessor)(input_layer)
    base_model = model_name(weights='imagenet', include_top=False,
                            input_shape=input_size)(preprocessor)
    avg = GlobalAveragePooling2D()(base_model)
    feature_extractor = Model(inputs = input_layer, outputs = avg)
    
    #Extract feature.
    feature_maps = feature_extractor.predict(data, verbose=1)
    print('Feature maps shape: ', feature_maps.shape)
    return feature_maps

In [22]:
# Extract features using InceptionV3 
from keras.applications.inception_v3 import InceptionV3, preprocess_input
inception_preprocessor = preprocess_input
inception_features = get_features(InceptionV3,
                                  inception_preprocessor,
                                  img_size, X_train)

224/224 [==============================] - 1021s 5s/step
Feature maps shape:  (7155, 2048)


In [47]:
del X #to free up some ram memory
gc.collect()

39713

In [23]:
#Creating final featuremap by combining all extracted features

final_features = np.concatenate([inception_features], axis=-1) #axis=-1 to concatinate horizontally

print('Final feature maps shape', final_features.shape)

Final feature maps shape (7155, 2048)


In [24]:
#Prepare Deep net

model = Sequential()
# model.add(Dense(1028,input_shape=(final_features.shape[1],)))
model.add(Dropout(0.7,input_shape=(final_features.shape[1],)))
model.add(Dense(n_classes,activation= 'softmax'))

model.compile(optimizer=adam,
              loss='categorical_crossentropy',
              metrics=['accuracy'])

#Training the model. 
history = model.fit(final_features, y_train,
            batch_size=batch_size,
            epochs=epochs,
            validation_split=0.2,
            callbacks=[lrr,EarlyStop])

Epoch 1/15
45/45 [==============================] - 2s 42ms/step - loss: 3.6068 - accuracy: 0.2930 - val_loss: 1.9031 - val_accuracy: 0.8344 - lr: 0.0010
Epoch 2/15
45/45 [==============================] - 0s 9ms/step - loss: 1.3700 - accuracy: 0.7904 - val_loss: 0.7751 - val_accuracy: 0.8917 - lr: 0.0010
Epoch 3/15
45/45 [==============================] - 0s 9ms/step - loss: 0.7023 - accuracy: 0.8726 - val_loss: 0.5119 - val_accuracy: 0.9022 - lr: 0.0010
Epoch 4/15
45/45 [==============================] - 0s 8ms/step - loss: 0.5056 - accuracy: 0.8868 - val_loss: 0.4203 - val_accuracy: 0.9050 - lr: 0.0010
Epoch 5/15
45/45 [==============================] - 0s 9ms/step - loss: 0.4062 - accuracy: 0.9034 - val_loss: 0.3685 - val_accuracy: 0.9126 - lr: 0.0010
Epoch 6/15
45/45 [==============================] - 0s 8ms/step - loss: 0.3533 - accuracy: 0.9139 - val_loss: 0.3425 - val_accuracy: 0.9154 - lr: 0.0010
Epoch 7/15
45/45 [==============================] - 0s 8ms/step - loss: 0.3142 - 

In [25]:
del final_features
gc.collect()

1554

In [27]:
#Extract test data features.
def extact_features(data):
    inception_features = get_features(InceptionV3, inception_preprocessor, img_size, data)
    final_features = np.concatenate([inception_features],axis=-1)
    
    print('Final feature maps shape', final_features.shape)
    
    #deleting to free up ram memory
    del inception_features
    gc.collect()
    
    return final_features

test_features = extact_features(X_test)

96/96 [==============================] - 446s 5s/step
Feature maps shape:  (3067, 2048)
Final feature maps shape (3067, 2048)


In [28]:
y_pred = model.predict(test_features)

96/96 [==============================] - 0s 2ms/step


In [42]:
from sklearn.metrics import accuracy_score

y_test_indices = np.argmax(y_test, axis=1)
y_pred_indices = np.argmax(y_pred, axis=1)

accuracy_score(y_test_indices, y_pred_indices)


0.9083795239647864